# FASE 1: IMPORTAR DEPENDENCIAS

In [2]:
!pip install tf-models-official==2.4.0

In [3]:
import tensorflow as tf

In [4]:
tf.__version__

'2.12.0'

In [6]:
import tensorflow_hub as hub

from official.nlp.bert.tokenization import FullTokenizer
from official.nlp.bert.input_pipeline import create_squad_dataset
from official.nlp.data.squad_lib import generate_tf_record_from_json_file

from official.nlp import optimization

from official.nlp.data.squad_lib import read_squad_examples
from official.nlp.data.squad_lib import FeatureWriter
from official.nlp.data.squad_lib import convert_examples_to_features
from official.nlp.data.squad_lib import write_predictions



In [7]:
import numpy as np
import math
import random
import time
import json
import collections
import os
from google.colab import drive

# Fase 2: Preprocesado de Datos

In [8]:
drive.mount("/content/drive")

Mounted at /content/drive


In [9]:
input_meta_data = generate_tf_record_from_json_file(
    "/content/drive/My Drive/BERT/train-v1.1.json",
    "/content/drive/My Drive/BERT/vocab.txt",
    "/content/drive/My Drive/BERT/train-v1.1.tf_record",
)

In [10]:
with tf.io.gfile.GFile("/content/drive/My Drive/BERT/train_meta_data", "w") as writer:
    writer.write(json.dumps(input_meta_data, indent=4) + "\n")


In [12]:
BATCH_SIZE = 4

train_dataset = create_squad_dataset(
    "/content/drive/My Drive/BERT/train-v1.1.tf_record",
    input_meta_data['max_seq_length'], # 384
    BATCH_SIZE,
    is_training=True)

# FASE 3: CONSTRUCCION DEL MODELO


CAPA SQUAD

In [13]:
class BertSquadLayer(tf.keras.layers.Layer):

  def __init__(self):
    super(BertSquadLayer, self).__init__()
    self.final_dense = tf.keras.layers.Dense(
        units=2,
        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))

  def call(self, inputs):
    logits = self.final_dense(inputs) # (batch_size, seq_len, 2)

    logits = tf.transpose(logits, [2, 0, 1]) # (2, batch_size, seq_len)
    unstacked_logits = tf.unstack(logits, axis=0) # [(batch_size, seq_len), (batch_size, seq_len)]
    return unstacked_logits[0], unstacked_logits[1]

# Modelo completo

In [14]:
class BERTSquad(tf.keras.Model):

    def __init__(self,
                 name="bert_squad"):
        super(BERTSquad, self).__init__(name=name)

        self.bert_layer = hub.KerasLayer(
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=True)

        self.squad_layer = BertSquadLayer()

    def apply_bert(self, inputs):
        _ , sequence_output = self.bert_layer([inputs["input_word_ids"],
                                               inputs["input_mask"],
                                               inputs["input_type_ids"]])
        return sequence_output

    def call(self, inputs):
        seq_output = self.apply_bert(inputs)

        start_logits, end_logits = self.squad_layer(seq_output)

        return start_logits, end_logits

# FASE 4: Entrenamiento

# Creacion de la IA

In [15]:
TRAIN_DATA_SIZE = 88641
NB_BATCHES_TRAIN = 2000
BATCH_SIZE = 4
NB_EPOCHS = 3
INIT_LR = 5e-5
WARMUP_STEPS = int(NB_BATCHES_TRAIN * 0.1)

In [16]:
train_dataset_light = train_dataset.take(NB_BATCHES_TRAIN)

In [18]:
bert_squad = BERTSquad()

In [19]:
optimizer = optimization.create_optimizer(
    init_lr=INIT_LR,
    num_train_steps=NB_BATCHES_TRAIN,
    num_warmup_steps=WARMUP_STEPS)

In [20]:
def squad_loss_fn(labels, model_outputs):
    start_positions = labels['start_positions']
    end_positions = labels['end_positions']
    start_logits, end_logits = model_outputs

    start_loss = tf.keras.backend.sparse_categorical_crossentropy(
        start_positions, start_logits, from_logits=True)
    end_loss = tf.keras.backend.sparse_categorical_crossentropy(
        end_positions, end_logits, from_logits=True)

    total_loss = (tf.reduce_mean(start_loss) + tf.reduce_mean(end_loss)) / 2

    return total_loss

train_loss = tf.keras.metrics.Mean(name="train_loss")

In [21]:
next(iter(train_dataset_light))

({'input_word_ids': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[ 101, 1999, 2054, ...,    0,    0,    0],
         [ 101, 2054, 2515, ...,    0,    0,    0],
         [ 101, 2129, 2116, ...,    0,    0,    0],
         [ 101, 1996, 2267, ...,    0,    0,    0]], dtype=int32)>,
  'input_mask': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
  'input_type_ids': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>},
 {'end_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([22, 95, 32, 48], dtype=int32)>,
  'start_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([22, 93, 29, 47], dtype=int32)>})

In [22]:
bert_squad.compile(optimizer,
                   squad_loss_fn)

In [23]:
checkpoint_path = "./drive/My Drive/BERT/ckpt_bert_squad/"

ckpt = tf.train.Checkpoint(bert_squad=bert_squad)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Último checkpoint restaurado!!")

Último checkpoint restaurado!!


# Entrenamiento personalizado

In [ ]:
for epoch in range(NB_EPOCHS):
    print("Inicio del Epoch {}".format(epoch+1))
    start = time.time()

    train_loss.reset_states()

    for (batch, (inputs, targets)) in enumerate(train_dataset_light):
        with tf.GradientTape() as tape:
            model_outputs = bert_squad(inputs)
            loss = squad_loss_fn(targets, model_outputs)

        gradients = tape.gradient(loss, bert_squad.trainable_variables)
        optimizer.apply_gradients(zip(gradients, bert_squad.trainable_variables))

        train_loss(loss)

        if batch % 50 == 0:
            print("Epoch {} Lote {} Pérdida {:.4f}".format(
                epoch+1, batch, train_loss.result()))

        if batch % 500 == 0:
            ckpt_save_path = ckpt_manager.save()
            print("Guardando checkpoint para el epoch {} en el directorio {}".format(epoch+1,
                                                                ckpt_save_path))
    print("Tiempo total para entrenar 1 epoch: {} segs\n".format(time.time() - start))

# Fase 5: Evaluación

# Preparando

In [ ]:
eval_examples = read_squad_examples(
    "/content/drive/My Drive/BERT/dev-v1.1.json",
    is_training=False,
    version_2_with_negative=False)

In [ ]:
eval_writer = FeatureWriter(
    filename=os.path.join("/content/drive/My Drive/BERT/",
                          "eval.tf_record"),
    is_training=False)

In [ ]:
my_bert_layer = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
    trainable=False)
vocab_file = my_bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = my_bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [ ]:
def _append_feature(feature, is_padding):
    if not is_padding:
        eval_features.append(feature)
    eval_writer.process_feature(feature)

In [ ]:
eval_features = []
dataset_size = convert_examples_to_features(
    examples=eval_examples,
    tokenizer=tokenizer,
    max_seq_length=384,
    doc_stride=128,
    max_query_length=64,
    is_training=False,
    output_fn=_append_feature,
    batch_size=4)

In [ ]:
eval_writer.close()

In [ ]:
BATCH_SIZE = 4

eval_dataset = create_squad_dataset(
    "/content/drive/My Drive/BERT/eval.tf_record",
    384,#input_meta_data['max_seq_length'],
    BATCH_SIZE,
    is_training=False)

# LLEVAR A CABO LAS PREDICCIONES

In [ ]:
RawResult = collections.namedtuple("RawResult",
                                   ["unique_id", "start_logits", "end_logits"])

In [ ]:
def get_raw_results(predictions):
    for unique_ids, start_logits, end_logits in zip(predictions['unique_ids'],
                                                    predictions['start_logits'],
                                                    predictions['end_logits']):
        yield RawResult(
            unique_id=unique_ids.numpy(),
            start_logits=start_logits.numpy().tolist(),
            end_logits=end_logits.numpy().tolist())

In [ ]:
all_results = []
for count, inputs in enumerate(eval_dataset):
    x, _ = inputs
    unique_ids = x.pop("unique_ids")
    start_logits, end_logits = bert_squad(x, training=False)
    output_dict = dict(
        unique_ids=unique_ids,
        start_logits=start_logits,
        end_logits=end_logits)
    for result in get_raw_results(output_dict):
        all_results.append(result)
    if count % 100 == 0:
        print("{}/{}".format(count, 2709))

In [ ]:
output_prediction_file = "/content/drive/My Drive/BERT/predictions.json"
output_nbest_file = "/content/drive/My Drive/BERT/nbest_predictions.json"
output_null_log_odds_file = "/content/drive/My Drive/BERT/null_odds.json"

write_predictions(
    eval_examples,
    eval_features,
    all_results,
    20,
    30,
    True,
    output_prediction_file,
    output_nbest_file,
    output_null_log_odds_file,
    verbose=False)

# FINISH